In [ ]:
import pandas as pd

# Your example dataframe
data = {
    'Account': ['Jon', 'Jon', 'Stan', 'Stan', 'Jon', 'Stan', 'Jon'],
    'Date': ['12/12/2023', '12/12/2023', '12/12/2023', '12/12/2023', '12/12/2023', '12/12/2023', '12/13/2023'],
    'Login': ['8:00:00', '', '14:00:00', '', '16:00:00', '15:00:00', '16:00:01'],
    'Logout': ['', '13:00:00', '', '15:00:00', '', '', ''],
    'Comp_num': [1, 1, 2, 2, 1, 1, 2]
}

df = pd.DataFrame(data)

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Sort dataframe by 'Account', 'Date', and 'Comp_num'
df = df.sort_values(by=['Account', 'Date', 'Comp_num'])

# Group by 'Account' and 'Comp_num' and concatenate 'Login' and 'Logout' times
df['Login'] = df.groupby(['Account', 'Comp_num'])['Login'].transform('first')
df['Logout'] = df.groupby(['Account', 'Comp_num'])['Logout'].transform('last')

# Drop duplicate rows keeping only the first occurrence
df = df.drop_duplicates(subset=['Account', 'Date', 'Comp_num'], keep='first')

# Reset index
df = df.reset_index(drop=True)

print(df)


In [ ]:
import pandas as pd

# Your example dataframe
data = {
    'Account': ['Jon', 'Jon', 'Stan', 'Stan', 'Jon', 'Stan', 'Jon'],
    'Date': ['12/12/2023', '12/12/2023', '12/12/2023', '12/12/2023', '12/12/2023', '12/12/2023', '12/13/2023'],
    'Login': ['8:00:00', '', '14:00:00', '', '16:00:00', '15:00:00', '16:00:01'],
    'Logout': ['', '13:00:00', '', '15:00:00', '', '', ''],
    'Comp_num': [1, 1, 2, 2, 1, 1, 2]
}

df = pd.DataFrame(data)

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Sort dataframe by 'Account', 'Date', and 'Comp_num'
df = df.sort_values(by=['Account', 'Date', 'Comp_num']).reset_index(drop=True)

# Create a new dataframe to store the modified rows
new_rows = []

for index, row in df.iterrows():
    if index == 0:
        current_account = row['Account']
        current_comp_num = row['Comp_num']
        login_time = row['Login']
        logout_time = row['Logout']
    else:
        if row['Account'] == current_account and row['Comp_num'] == current_comp_num:
            if row['Login']:
                login_time = min(pd.to_datetime(row['Login']), pd.to_datetime(login_time))
            if row['Logout']:
                logout_time = max(pd.to_datetime(row['Logout']), pd.to_datetime(logout_time))
        else:
            new_rows.append([current_account, row['Date'], login_time, logout_time, current_comp_num])
            current_account = row['Account']
            current_comp_num = row['Comp_num']
            login_time = row['Login']
            logout_time = row['Logout']

# Add the last gathered data to the new dataframe
new_rows.append([current_account, row['Date'], login_time, logout_time, current_comp_num])

# Create a new dataframe from the modified rows
new_df = pd.DataFrame(new_rows, columns=['Account', 'Date', 'Login', 'Logout', 'Comp_num'])

print(new_df)


In [ ]:
import pandas as pd

# Your example dataframe
data = {
    'Account': ['Jon', 'Jon', 'Stan', 'Stan', 'Jon', 'Stan', 'Jon'],
    'Date': ['12/12/2023', '12/12/2023', '12/12/2023', '12/12/2023', '12/12/2023', '12/12/2023', '12/13/2023'],
    'Login': ['8:00:00', '', '14:00:00', '', '16:00:00', '15:00:00', '16:00:01'],
    'Logout': ['', '13:00:00', '', '15:00:00', '', '', ''],
    'Comp_num': [1, 1, 2, 2, 1, 1, 2]
}

df = pd.DataFrame(data)

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Sort dataframe by 'Account', 'Date', and 'Comp_num'
df = df.sort_values(by=['Account', 'Date', 'Comp_num'])

# Initialize new DataFrame to store modified data
new_data = {'Account': [], 'Date': [], 'Login': [], 'Logout': [], 'Comp_num': []}

# Iterate through rows
for index, row in df.iterrows():
    if row['Login']:  # If Login time is present
        if not new_data['Account'] or new_data['Account'][-1] != row['Account'] or new_data['Comp_num'][-1] != row['Comp_num']:
            # If new Account or Comp_num encountered, append new row
            new_data['Account'].append(row['Account'])
            new_data['Date'].append(row['Date'])
            new_data['Login'].append(row['Login'])
            new_data['Comp_num'].append(row['Comp_num'])
            new_data['Logout'].append('')  # Set Logout time as empty for now
        else:
            # If same Account and Comp_num, update the corresponding row with Logout time
            idx = len(new_data['Account']) - 1
            new_data['Logout'][idx] = row['Login']  # Assign Login time as Logout time

# Create the modified DataFrame
modified_df = pd.DataFrame(new_data)

print(modified_df)


In [ ]:
import matplotlib.pyplot as plt

# Assuming 'new_df' contains the relevant data
# Convert 'Date' column to datetime if it's not already in datetime format
new_df['Date'] = pd.to_datetime(new_df['Date'])

# Sort dataframe by 'Date'
new_df = new_df.sort_values(by='Date')

# Plotting login patterns over time
plt.figure(figsize=(12, 6))
plt.plot(new_df['Date'], new_df.index, marker='o', linestyle='-', color='b')
plt.title('Login Patterns Over Time')
plt.xlabel('Date')
plt.ylabel('Login Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'new_df' contains the relevant data
# Convert 'Date' column to datetime if it's not already in datetime format
new_df['Date'] = pd.to_datetime(new_df['Date'])

# Sort dataframe by 'Date'
new_df = new_df.sort_values(by='Date')

# Plotting login patterns over time using Seaborn lineplot
plt.figure(figsize=(12, 6))
sns.lineplot(x='Date', y=new_df.index, data=new_df, marker='o')
plt.title('Login Patterns Over Time')
plt.xlabel('Date')
plt.ylabel('Login Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
